## Notebook rilut

bikin jelek: colorCode, total_spent, productGroup (kalau di-impute paksa dengan 0)

In [39]:
from __future__ import division

label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from operator import itemgetter
#from sklearn.externals import joblib

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

def feature_importances(Xdf, rf_model):
    pair=zip(Xdf.columns, rf_model.feature_importances_)
    t1=[i for i in pair]; t1
    return sorted(t1, key=itemgetter(1))

In [40]:
# Pakai make_datasets_train_only. Buat sendiri juga boleh

df=pd.read_csv('train_gue.csv')

if 'deviceID' in df.columns:
    del df['deviceID']

df.head()

,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherAmount,...,customer_budget,after_voucher,order_order,choice_order,articleID_prob,colorCode_prob,customerID_prob,sizeCode_prob,ac_prob,as_prob
0,a1000001,2014-01-01,i1000382,1972,44,3.0,1,10.0,29.984375,0.0,...,28.500000,30.0,1,1,0.421143,0.521484,0.0,0.516113,0.421053,0.421053
1,a1000001,2014-01-01,i1000550,3854,44,3.0,1,20.0,40.000000,0.0,...,28.500000,30.0,2,2,0.506836,0.492432,0.0,0.516113,0.475410,0.475410
2,a1000002,2014-01-01,i1001991,2974,38,8.0,1,35.0,50.000000,0.0,...,84.989998,85.0,1,1,0.642090,0.455322,0.5,0.535645,0.616505,0.616505
3,a1000002,2014-01-01,i1001999,1992,38,8.0,1,50.0,50.000000,0.0,...,84.989998,85.0,2,2,0.694336,0.479492,0.5,0.535645,0.713450,0.713450
4,a1000003,2014-01-01,i1001942,1968,42,8.0,1,10.0,36.000000,0.0,...,60.000000,60.0,1,1,0.600586,0.606934,0.0,0.526367,0.573034,0.573034


In [80]:
# Metodeku

total_spent_dict = df[['customerID', 'order_total']].groupby('customerID').sum()['order_total'].to_dict()
df['total_spent'] = df.customerID.apply(total_spent_dict.get).astype(np.float32)
del total_spent_dict

In [64]:
df['yearly_expense'] = (df['customer_budget'] / df['total_spent']).astype(np.float16)

In [43]:
#Konversi kategori/object ke angka (ordinal)
print "Konversi kategori/object ke angka (ordinal):"

#Cari kolom yang tipenya object, bukan integer maupun float
object_columns=df.loc[:, df.dtypes == object].columns

for col in object_columns:
    print col
    le=LabelEncoder()
    # Konversi deh
    df[col]=le.fit_transform(df[col])
    
if 'orderID' in df.columns:
    del df['orderID']

Konversi kategori/object ke angka (ordinal):
orderID
orderDate
articleID
sizeCode
customerID
paymentMethod
mmdd


In [78]:
# Turunin presisi, iseng-iseng aja
float_64_columns = df.loc[:, df.dtypes == np.float64].columns
for col in float_64_columns:
    df[col] = df[col].astype(np.float16)

int_64_columns = df.loc[:, df.dtypes == np.int64].columns
for col in int_64_columns:
    df[col] = df[col].astype(np.int32)
    

# Impute
df.rrp.fillna(df.rrp.median(), inplace=True)
df.productGroup.fillna(0, inplace=True)


Xdf=df.drop([label, "colorCode",'customerID','productGroup','total_spent'], axis=1)
print Xdf.columns.tolist()
X=Xdf.values
y=df.returnQuantity.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

['orderDate', 'articleID', 'sizeCode', 'quantity', 'price', 'rrp', 'voucherAmount', 'paymentMethod', 'mmdd', 'months', 'order_total', 'customer_budget', 'after_voucher', 'order_order', 'choice_order', 'articleID_prob', 'colorCode_prob', 'customerID_prob', 'sizeCode_prob', 'ac_prob', 'as_prob', 'yearly_expense']


In [79]:
lr=LinearRegression(n_jobs=-1, normalize=True)
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[116936  50715     21      0      0      0]
 [ 31834 148676     14      0      0      0]
 [     5    514     26      0      0      0]
 [     0     16     10      0      0      0]
 [     0      0      3      0      0      0]
 [     0      0      4      1      0      0]]
83191


# 83191

In [63]:
df.head()

,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherAmount,customerID,...,order_order,choice_order,articleID_prob,colorCode_prob,customerID_prob,sizeCode_prob,ac_prob,as_prob,total_spent,yearly_expense
0,0,346,1972,16,3.0,1,10.0,29.984375,0.0,9962,...,1,1,0.421143,0.521484,0.0,0.516113,0.421143,0.421143,114.000000,0.250000
1,0,511,3854,16,3.0,1,20.0,40.000000,0.0,9962,...,2,2,0.506836,0.492432,0.0,0.516113,0.475342,0.475342,114.000000,0.250000
2,0,1901,2974,13,8.0,1,35.0,50.000000,0.0,42755,...,1,1,0.642090,0.455322,0.5,0.535645,0.616699,0.616699,169.979996,0.500000
3,0,1909,1992,13,8.0,1,50.0,50.000000,0.0,42755,...,2,2,0.694336,0.479492,0.5,0.535645,0.713379,0.713379,169.979996,0.500000
4,0,1852,1968,15,8.0,1,10.0,36.000000,0.0,82469,...,1,1,0.600586,0.606934,0.0,0.526367,0.573242,0.573242,300.000000,0.199951


In [47]:
# rfr=RandomForestRegressor(20, n_jobs=3)
# rfr.fit(X_train, y_train)
# y_pred_rfr=rfr.predict(X_test)
# y_pred_rfr=np.array([round(yp) for yp in y_pred_rfr], dtype=np.int8)
# y_pred_rfr[y_pred_rfr<0]=0
# print confusion_matrix(y_test,y_pred_rfr)
# print dmc_error(y_test,y_pred_rfr)

In [48]:
# feature_importances(Xdf, rfr)

In [49]:
# y_pred_ens=(1*rfr.predict(X_test) + 9*model.predict(X_test))/10
# y_pred_ens=np.array([round(yp) for yp in y_pred_ens], dtype=np.int8)
# y_pred_ens[y_pred_ens<0]=0
# print confusion_matrix(y_test,y_pred_ens)
# print dmc_error(y_test,y_pred_ens)

```

```